In [1]:
import os
import copy
import time
import pickle
import numpy as np
import easydict
from tqdm import tqdm

import torch
from tensorboardX import SummaryWriter

from options import args_parser
from update import LocalUpdate, test_inference
from models import MLP, CNNMnist, CNNFashion_Mnist, CNNCifar, CNNCifar_fedVC, CNNCifar_VCBN, CNNCifar_VCGN, CNNCifar_WS
from utils import get_dataset, average_weights, exp_details, get_logger, check_norm
from fed_cifar100 import load_partition_data_federated_cifar100
from resnet_gn import resnet18
from resnet import ResNet32_test
from vgg import vgg11_bn, vgg11
import random
import logging
import datetime
from test import *


In [2]:
args = easydict.EasyDict({
    "model": 'resnet18_ws',
    'dataset': 'cifar100',
    'gpu': 0,
    'iid': 2,
    'epochs': 1000,
    'optimizer': 'clip',
    'seed': 0,
    'norm': 'group_norm',
    'num_users': 500,
    'frac': 0.1,
    'local_ep': 1,
    'local_bs': 20,
    'lr': 1,
    'momentum': 0.9,
    'kernel_num': 9,
    'kernel_sizes': '3,4,5',
    'num_channnels': '1',
    'num_filters': 32,
    'max_pool': 'True',
    'num_classes': 100,
    'unequal': 0,
    'stopping_rounds': 10,
    'verbose': 0,
    'hold_normalize':0
})

train_loss, val_acc_list, batch_loss_list, conv_grad_list, fc_grad_list, total_grad_list= main_test(args)

03/05 05:16:56 PM | {'model': 'resnet18_ws', 'dataset': 'cifar100', 'gpu': 0, 'iid': 2, 'epochs': 1000, 'optimizer': 'clip', 'seed': 0, 'norm': 'group_norm', 'num_users': 500, 'frac': 0.1, 'local_ep': 1, 'local_bs': 20, 'lr': 1, 'momentum': 0.9, 'kernel_num': 9, 'kernel_sizes': '3,4,5', 'num_channnels': '1', 'num_filters': 32, 'max_pool': 'True', 'num_classes': 100, 'unequal': 0, 'stopping_rounds': 10, 'verbose': 0, 'hold_normalize': 0}
INFO:gal:{'model': 'resnet18_ws', 'dataset': 'cifar100', 'gpu': 0, 'iid': 2, 'epochs': 1000, 'optimizer': 'clip', 'seed': 0, 'norm': 'group_norm', 'num_users': 500, 'frac': 0.1, 'local_ep': 1, 'local_bs': 20, 'lr': 1, 'momentum': 0.9, 'kernel_num': 9, 'kernel_sizes': '3,4,5', 'num_channnels': '1', 'num_filters': 32, 'max_pool': 'True', 'num_classes': 100, 'unequal': 0, 'stopping_rounds': 10, 'verbose': 0, 'hold_normalize': 0}


{'model': 'resnet18_ws', 'dataset': 'cifar100', 'gpu': 0, 'iid': 2, 'epochs': 1000, 'optimizer': 'clip', 'seed': 0, 'norm': 'group_norm', 'num_users': 500, 'frac': 0.1, 'local_ep': 1, 'local_bs': 20, 'lr': 1, 'momentum': 0.9, 'kernel_num': 9, 'kernel_sizes': '3,4,5', 'num_channnels': '1', 'num_filters': 32, 'max_pool': 'True', 'num_classes': 100, 'unequal': 0, 'stopping_rounds': 10, 'verbose': 0, 'hold_normalize': 0}


03/05 05:17:23 PM | ResNet(
  (conv1): Conv2d_stand(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): GroupNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d_stand(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d_stand(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): GroupNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (relu): ReLU(inplace=True)
      (bn2): GroupNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
    )
    (1): BasicBlock(
      (conv1): Conv2d_stand(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d_stand(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1

num_channels_per_group:32
num_channels_per_group:32
num_channels_per_group:32
num_channels_per_group:32
num_channels_per_group:32
num_channels_per_group:64
num_channels_per_group:64
num_channels_per_group:64
num_channels_per_group:64
num_channels_per_group:64
num_channels_per_group:128
num_channels_per_group:128
num_channels_per_group:128
num_channels_per_group:128
num_channels_per_group:128
num_channels_per_group:256
num_channels_per_group:256
num_channels_per_group:256
num_channels_per_group:256
num_channels_per_group:256
ResNet(
  (conv1): Conv2d_stand(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): GroupNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d_stand(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Con

  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]

...


 | Global Training Round : 569 |



03/07 08:54:48 PM |  
Avg Training Stats after 569 global rounds:
INFO:gal: 
Avg Training Stats after 569 global rounds:
03/07 08:54:48 PM | Training Loss : 2.35318550992012
INFO:gal:Training Loss : 2.35318550992012
03/07 08:54:48 PM | Train Accuracy: 30.71% 

INFO:gal:Train Accuracy: 30.71% 

 57%|████████████████████████████████████████▉                               | 569/1000 [51:37:18<88:38:58, 740.46s/it]


 | Global Training Round : 570 |



 57%|████████████████████████████████████████▉                               | 569/1000 [51:44:59<39:11:56, 327.42s/it]


KeyboardInterrupt: 